In [ ]:
import numpy as np
import pandas as pd
import time

import random

import matplotlib
from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_probability as tfp
from tqdm import tqdm

import pickle5 as pickle
from scipy import stats

import matplotlib.ticker as tick

import sys
sys.path.append('..')
from nsgp_vi import nsgpVI

# We'll use double precision throughout for better numerics.
dtype = np.float64

tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels

plt.style.use('ggplot') 
plt.style.use('seaborn-paper')
plt.style.use('seaborn-whitegrid')


In [ ]:
print(tfp.__version__)
print(tf.__version__)

In [ ]:
df = pd.read_csv('../data/power.csv', thousands=',')

dates = pd.DatetimeIndex(df['Date'],dayfirst=True)#.values
times = pd.DatetimeIndex(df['Time'])

T = (24*(dates.day-1) + times.hour  + times.minute/60.).astype(float).values[:,None]#(time in hours)
X = df['Global_active_power'].astype('float').values.reshape(len(T),1)

meanX = np.mean(X)
X = X - meanX
X = X/X.std()

print(T.shape)
print(X.shape)

In [ ]:
minutes = (60.*times.hour  + times.minute).astype(float).values
gap = X[:,0]

bin_means, bin_edges, binnumber = stats.binned_statistic(minutes, gap, statistic='mean', bins=48)
bin_width = (bin_edges[1] - bin_edges[0])
bin_centers = bin_edges[1:] - bin_width/2

plt.figure()
plt.plot(bin_centers/60., bin_means)
plt.legend(fontsize=10)
plt.show()

In [ ]:
num_training_points_ = T.shape[0]

num_inducing_points_ = 48

inducing_index_points = np.linspace(0., 24., num_inducing_points_, endpoint=False)[..., np.newaxis]
np.random.shuffle(inducing_index_points)


In [ ]:
plt.plot(T,X,'.',markersize=5)
plt.xlabel('Time (hours)')
plt.ylabel('Std. obs. (Global active power in killowatts)')
plt.tick_params(axis='both', which='major', labelsize=10)
plt.savefig("Power_data_vi.pdf",dpi=600)
plt.show()

In [ ]:
BATCH_SIZE=8
SEG_LENGTH=1024


class segment_generator:
    def __iter__(self):
        
        # loop over segments
        self.j = 0
        self.max_j = num_training_points_//SEG_LENGTH
        
        
        return self

    def __next__(self):
        

        if self.j==self.max_j:
            raise StopIteration

        TT = T[self.j*SEG_LENGTH:(self.j+1)*SEG_LENGTH]
        XX = X[self.j*SEG_LENGTH:(self.j+1)*SEG_LENGTH]
    
        self.j += 1

        return TT,XX

        
dataset = tf.data.Dataset.from_generator(segment_generator, (tf.float64)) 
dataset = dataset.map(lambda dd: (dd[0],dd[1]))
dataset = dataset.shuffle(1000)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
dataset = tf.data.Dataset.from_generator(segment_generator, (tf.float64)) 
dataset = dataset.map(lambda dd: (dd[0],dd[1]))
dataset = dataset.shuffle(1000)
dataset = dataset.batch(BATCH_SIZE)

In [ ]:
for d in dataset:
    print(d[0].shape,d[1].shape)

In [ ]:
kernel_len_a = tfp.util.TransformedVariable(1.0, tfb.Softplus(),dtype=tf.float64, name='k_len_a',trainable=True)
kernel_len_l = tfp.util.TransformedVariable(1.0, tfb.Chain([tfb.Shift(np.float64(0.5)), tfb.Softplus()]),dtype=tf.float64, name='k_len_l',trainable=True)

# amplitude kernel parameters, lower levels
kernel_amp_a = tfp.util.TransformedVariable(1.0, tfb.Softplus(),dtype=tf.float64, name='k_amp_a',trainable=True)
kernel_amp_l = tfp.util.TransformedVariable(1.0, tfb.Chain([tfb.Shift(np.float64(0.5)), tfb.Softplus()]),dtype=tf.float64, name='k_amp_l',trainable=True)

#kernels on the second layer
kernel_len = tfk.ExpSinSquared(kernel_len_a,kernel_len_l,period=np.float64(24.0))
kernel_amp = tfk.ExpSinSquared(kernel_amp_a,kernel_amp_l,period=np.float64(24.0))

vgp = nsgpVI(kernel_len,kernel_amp,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=20,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  


In [ ]:
#loss = vgp.optimize(BATCH_SIZE, SEG_LENGTH, NUM_EPOCHS=500)


In [ ]:
#Load the inducing points and the optimized parameters 
with open('../data/power-results/power.pkl', 'rb') as f:
    loadp = pickle.load(f)
    
inducing_index_points = np.load('../data/power-results/T_ind_power.npy')    

In [ ]:
vgp = nsgpVI(kernel_len,kernel_amp,n_inducing_points=num_inducing_points_,inducing_index_points=inducing_index_points,dataset=dataset,num_training_points=num_training_points_, num_sequential_samples=20,num_parallel_samples=10,init_observation_noise_variance=0.005**2)  


In [ ]:
#Load the parameters !
for np_v, tf_v in zip(loadp,vgp.trainable_variables):
    tf_v.assign(np_v)

In [ ]:

ZZ = np.linspace(0,24,200)[:,None]

[len_mean,amp_mean], [len_var,amp_var] = vgp.get_marginal(ZZ[None,...])

len_mean = len_mean[0,:,0].numpy()
len_std = len_var[:,0].numpy()**0.5

amp_mean = amp_mean[0,:,0].numpy()
amp_std = amp_var[:,0].numpy()**0.5


f, (ax1, ax2) = plt.subplots(1, 2,figsize=(16,6))


ax1.plot(ZZ,tf.math.softplus(vgp.mean_len + len_mean),color='C1')
ax1.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.28*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.28*len_std),color='C1',alpha=0.25)
ax1.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 1.96*len_std),tf.math.softplus(vgp.mean_len + len_mean + 1.96*len_std),color='C1',alpha=0.25)
ax1.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_len + len_mean - 2.58*len_std),tf.math.softplus(vgp.mean_len + len_mean + 2.58*len_std),color='C1',alpha=0.25)

ax1.set_xlabel('Time',size=15)
ax1.set_ylabel('Average power consumption persistence',size=15)
ax1.text(-0.05,1,'A', size=20, transform=ax1.transAxes)
ax1.tick_params(axis='both', which='major', labelsize=15)
tick_locator = tick.MaxNLocator(nbins=7)

ax2.plot(ZZ,tf.math.softplus(vgp.mean_amp + amp_mean),color='C1')
ax2.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.28*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.28*amp_std),color='C1',alpha=0.25)
ax2.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 1.96*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 1.96*amp_std),color='C1',alpha=0.25)
ax2.fill_between(ZZ[:,0],tf.math.softplus(vgp.mean_amp + amp_mean - 2.58*amp_std),tf.math.softplus(vgp.mean_amp + amp_mean + 2.58*amp_std),color='C1',alpha=0.25)

ax2.set_xlabel('Time',size=15)
ax2.set_ylabel('Average power consumption variance',size=15)
ax2.text(-0.1,1,'B', size=20, transform=ax2.transAxes)
ax2.tick_params(axis='both', which='major', labelsize=15)
tick_locator = tick.MaxNLocator(nbins=7)

plt.savefig("power_inference_vi.pdf",dpi=600)

plt.show()
